# Notebook: DB SQL Queries for Web/Front-end Team

## **Setup:**

In [107]:
# Import libraries we will use:
from dotenv import load_dotenv
import os
import pandas as pd
import psycopg2

In [108]:
# Get access info from .env file:
load_dotenv()

# PostgreSQL DB info:
PG_DB_HOST = os.getenv("PG_DB_HOST")
PG_DB_PORT = os.getenv("PG_DB_PORT")
PG_DB_NAME = os.getenv("PG_DB_NAME")
PG_DB_USER = os.getenv("PG_DB_USER")
PG_DB_PW = os.getenv("PG_DB_PW")
PG_DB_URI = os.getenv("PG_DB_URI")

In [109]:
# Open a connection to our PostgreSQL DB:
pg_conn = psycopg2.connect(
    host = PG_DB_HOST,
    port = PG_DB_PORT,
    database = PG_DB_NAME,
    user = PG_DB_USER,
    password = PG_DB_PW
)

# Instantiate a cursor using this connection:
pg_cursor = pg_conn.cursor()

## **Functions: Get Info From Our PostgreSQL DB:**

In [222]:
def get_feedback_for_user(user_id:int):
    """
    Using the user's user_id in our database (= 'owner_id' in the 'feedback' table),
    looks up and returns all feedback on all of that user's videos.
    """

    # Check to make sure input param video_id is the right type:
    if type(user_id) is not int:
        raise ValueError('Invalid user_id')

    # Get all feedback for the given video from our PostgreSQL DB:
    pg_cursor.execute(f"SELECT fb.id, fb.post, fb.video_id, fb.created_at, fb.updated_at, fb.overall_performance, fb.delivery_and_presentation, fb.response_quality, fb.audio_quality, fb.visual_environment FROM feedback as fb, videos as vds WHERE (fb.video_id = vds.id AND vds.owner_id={user_id});")
    column_names = [column_name[0] for column_name in pg_cursor.description]

    feedback_dataframe = pd.DataFrame(data=pg_cursor.fetchall(), columns=column_names)

    return feedback_dataframe

In [223]:
def get_feedback_for_video(video_id:int):
    """
    Using the video's video_id in our database, looks up and
    returns all feedback on that video.
    """

#     # Check to make sure input param video_id is the right type:
#     if type(video_id) is not int:
#         raise ValueError('Invalid video_id')

    # Get all feedback for the given video from our PostgreSQL DB:
    pg_cursor.execute(f"SELECT * FROM feedback WHERE video_id={video_id};")
    column_names = [column_name[0] for column_name in pg_cursor.description]

    feedback_dataframe = pd.DataFrame(data=pg_cursor.fetchall(), columns=column_names)

    return feedback_dataframe

In [224]:
def get_video_info(video_s3_key:str):
    """
    Using the video's S3 key, looks up info for that video, user and prompt
    in our PostgreSQL database, and returns all info in a dictionary with
    the following top-level keys:
    {
        'video': {
            info for the video itself
        },
        'prompt': {
            info for the prompt the video was responding to
        },
        'user': {
            includes info for the user who posted the video
        }
    }
    """

    # Return error if the input video_s3_key param is not a string:
    if type(video_s3_key) is not str:
        raise ValueError('Invalid video_s3_key')

    video_info = {'video': {},
                  'prompt': {},
                  'user': {}}

    # Lookup info for this video and user in our DB:

    # From DB table: 'videos'
    lookup_table_videos = lookup_in_videos_table(video_s3_key)
    if lookup_table_videos != "No data for this key in this table.":
        # Video info: Add video info to the video_info dict we will return:
        video_info['video']['video_id'] = lookup_table_videos['id']
        video_info['video']['title'] = lookup_table_videos['title']
        s3_key = lookup_table_videos['video_url']
        video_info['video']['s3_key'] = s3_key
        video_info['video']['s3_filename'] = s3_key.split('/')[-1]
        video_info['video']['created_at'] = lookup_table_videos['created_at']
        video_info['video']['updated_at'] = lookup_table_videos['updated_at']

        video_info['user']['user_id'] = lookup_table_videos['owner_id']
        video_info['prompt']['prompt_id'] = lookup_table_videos['prompt_id']

        # From DB table: 'users'
        lookup_table_users = lookup_in_users_table(video_info['user']['user_id'])
        if lookup_table_users != "No data for this key in this table.":
            # User info: Add user info to the video_info dict we will return:
            first_name = lookup_table_users['first_name'].capitalize()
            last_name = lookup_table_users['last_name'].capitalize()
            video_info['user']['first_name'] = first_name
            video_info['user']['last_name'] = last_name
            video_info['user']['name'] = first_name + " " + last_name
            video_info['user']['username'] = lookup_table_users['username']

        # From DB table: 'prompts'
        lookup_table_prompts = lookup_in_prompts_table(video_info['prompt']['prompt_id'])
        if lookup_table_prompts != "No data for this key in this table.":
            # Prompt info: Add prompt/question info to the the video_info dict:
            video_info['prompt']['question'] = lookup_table_prompts['question']

    return video_info

In [225]:
def lookup_in_videos_table(video_s3_key):
    """
    Using an S3 key for a video (e.g., from a message in our SQS queue),
    get the info for this video from our DB table 'videos' and return it as a dict.
    """

    pg_cursor.execute(f"SELECT * FROM videos WHERE video_url='{video_s3_key}';")
    results = pg_cursor.fetchall()

    # Check to make sure results are not empty (i.e., no data in this table):
    if results == []:
        return "No data for this key in this table."

    # If there is data for this user_id in this table, return it:
    values = results[0]
    column_names = [column_name[0] for column_name in pg_cursor.description]
    results = dict(zip(column_names, values))

    return results

In [226]:
def lookup_in_users_table(user_id):
    """
    Using a user_id in our TeamReel production DB (= owner_id in the 'users' table),
    get the info for this user from our DB table 'users' and return it as a dict.
    """

    pg_cursor.execute(f"SELECT * FROM users WHERE id={user_id};")
    results = pg_cursor.fetchall()

    # Check to make sure results are not empty (i.e., no data in this table):
    if results == []:
        return "No data for this key in this table."

    # If there is data for this user_id in this table, return it:
    values = results[0]
    column_names = [column_name[0] for column_name in pg_cursor.description]
    results = dict(zip(column_names, values))

    return results

In [227]:
def lookup_in_prompts_table(prompt_id):
    """
    Using a user_id in our TeamReel production DB (= owner_id in the 'users' table),
    get the info for this user from our DB table 'users' and return it as a dict.
    """

    pg_cursor.execute(f"SELECT * FROM prompts WHERE id={prompt_id};")
    results = pg_cursor.fetchall()

    # Check to make sure results are not empty (i.e., no data in this table):
    if results == []:
        return "No data for this key in this table."

    # If there is data for this user_id in this table, return it:
    values = results[0]
    column_names = [column_name[0] for column_name in pg_cursor.description]
    results = dict(zip(column_names, values))

    return results

## Testing Our Functions:

In [228]:
user_id = 201
video_id = 164

In [229]:
# get_feedback_for_user(user_id:int):
get_feedback_for_user(user_id=201)

,id,post,video_id,created_at,updated_at,viewed,overall_performance,delivery_and_presentation,response_quality,audio_quality,visual_environment
0,182,Great -- no comments!,164,2020-06-10 23:01:14.616412+00:00,2020-06-10 23:01:14.616412+00:00,False,2,1,1,2,2
1,183,Great -- no comments!,164,2020-06-02 23:01:14.616412+00:00,2020-06-02 23:01:14.616412+00:00,False,3,1,2,3,1
2,184,Great -- no comments!,164,2020-06-06 23:01:14.616412+00:00,2020-06-06 23:01:14.616412+00:00,False,3,1,2,4,4
3,185,Great -- no comments!,164,2020-06-09 23:01:14.616412+00:00,2020-06-09 23:01:14.616412+00:00,False,2,1,2,4,2
4,186,Great -- no comments!,164,2020-06-06 23:01:14.616412+00:00,2020-06-06 23:01:14.616412+00:00,False,4,2,1,4,2
5,187,Great -- no comments!,165,2020-06-08 23:01:14.616412+00:00,2020-06-08 23:01:14.616412+00:00,False,2,4,2,3,2
6,188,Great -- no comments!,165,2020-06-03 23:01:14.616412+00:00,2020-06-03 23:01:14.616412+00:00,False,1,2,4,1,3
7,189,Great -- no comments!,165,2020-06-03 23:01:14.616412+00:00,2020-06-03 23:01:14.616412+00:00,False,3,4,3,1,2
8,190,Great -- no comments!,165,2020-06-06 23:01:14.616412+00:00,2020-06-06 23:01:14.616412+00:00,False,4,3,4,3,4
9,191,Great -- no comments!,165,2020-06-11 23:01:14.616412+00:00,2020-06-11 23:01:14.616412+00:00,False,4,4,1,3,4


In [230]:
# get_feedback_for_video(video_id:int):
get_feedback_for_video(video_id=164)

,id,post,video_id,owner_id,created_at,updated_at,viewed,overall_performance,delivery_and_presentation,response_quality,audio_quality,visual_environment
0,182,Great -- no comments!,164,215,2020-06-10 23:01:14.616412+00:00,2020-06-10 23:01:14.616412+00:00,False,2,1,1,2,2
1,183,Great -- no comments!,164,216,2020-06-02 23:01:14.616412+00:00,2020-06-02 23:01:14.616412+00:00,False,3,1,2,3,1
2,184,Great -- no comments!,164,191,2020-06-06 23:01:14.616412+00:00,2020-06-06 23:01:14.616412+00:00,False,3,1,2,4,4
3,185,Great -- no comments!,164,215,2020-06-09 23:01:14.616412+00:00,2020-06-09 23:01:14.616412+00:00,False,2,1,2,4,2
4,186,Great -- no comments!,164,215,2020-06-06 23:01:14.616412+00:00,2020-06-06 23:01:14.616412+00:00,False,4,2,1,4,2


## Take a Look at the Relevant Tables in Our DB:

In [233]:
def get_db_table_as_dataframe(table_name):
    """
    Return the specified table in our database as a Pandas dataframe.
    """
    # Get info from the specified table: All rows (fetchall()):
    pg_cursor.execute(f"SELECT * FROM {table_name}")
    column_names = [column_name[0] for column_name in pg_cursor.description]
    
    table_as_dataframe = pd.DataFrame(data=pg_cursor.fetchall(), columns=column_names)
    
    return table_as_dataframe

In [244]:
# Get the names of all tables in our DB:
pg_cursor.execute(
    """
    SELECT table_name
    FROM information_schema.tables
    WHERE table_schema = 'public';
    """
)

db_table_names = sorted([table_name[0] for table_name in pg_cursor.fetchall()])

irrelevant_here = ['avatars', 'knex_migrations', 'knex_migrations_lock', 
                   'organizations', 'organizations_users', 'roles', 
                   'team_invite_link'
                  ]

for table_name in irrelevant_here:
    db_table_names.remove(table_name)

db_table_names

['feedback',
 'prompts',
 'team_members',
 'teams',
 'users',
 'videos',
 'videos_feedback']

In [245]:
index = 0
table_name = db_table_names[index]

In [246]:
print(f"\nDATABASE TABLE: '{table_name.upper()}' \n")
get_db_table_as_dataframe(table_name)


DATABASE TABLE: 'FEEDBACK' 



,id,post,video_id,owner_id,created_at,updated_at,viewed,overall_performance,delivery_and_presentation,response_quality,audio_quality,visual_environment
0,146,All fives,163,197,2020-06-12 23:16:20.295469+00:00,2020-06-12 23:16:20.295469+00:00,False,5,5,5,5,5
1,179,Great -- no comments!,167,227,2020-06-03 23:01:14.616412+00:00,2020-06-03 23:01:14.616412+00:00,False,2,1,4,2,4
2,180,Great -- no comments!,167,201,2020-06-10 23:01:14.616412+00:00,2020-06-10 23:01:14.616412+00:00,False,2,1,2,1,2
3,181,Great -- no comments!,167,197,2020-06-02 23:01:14.616412+00:00,2020-06-02 23:01:14.616412+00:00,False,1,1,1,2,2
4,182,Great -- no comments!,164,215,2020-06-10 23:01:14.616412+00:00,2020-06-10 23:01:14.616412+00:00,False,2,1,1,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...
76,177,Great -- no comments!,167,198,2020-06-05 23:01:14.616412+00:00,2020-06-05 23:01:14.616412+00:00,False,1,2,4,3,2
77,178,Great -- no comments!,167,226,2020-06-06 23:01:14.616412+00:00,2020-06-06 23:01:14.616412+00:00,False,2,1,1,4,4
78,204,Great -- no comments!,162,197,2020-06-05 23:01:14.616412+00:00,2020-06-05 23:01:14.616412+00:00,False,4,1,4,4,4
79,224,Good eye contact!,161,200,2020-06-13 01:48:19.068522+00:00,2020-06-13 01:48:19.068522+00:00,False,5,4,5,4,3


In [247]:
index +=1
table_name = db_table_names[index]

In [248]:
print(f"\nDATABASE TABLE: '{table_name.upper()}' \n")
get_db_table_as_dataframe(table_name)


DATABASE TABLE: 'PROMPTS' 



,id,question,description,team_id,created_at,updated_at
0,1,Tell me a little about yourself.,Maecenas pulvinar lobortis est. Phasellus sit ...,1,2019-03-07 01:33:02+00:00,2019-04-22 13:16:51+00:00
1,2,Why did you decide to get into _track_ and bec...,Suspendisse ornare consequat lectus.,1,2019-06-26 23:51:56+00:00,2019-08-12 09:21:58+00:00
2,3,Tell me about the most recent project you work...,Praesent blandit lacinia erat. Praesent blandi...,2,2019-10-02 12:56:37+00:00,2020-01-10 06:40:48+00:00
3,4,Desribe a time you were able to improve upon t...,Vivamus in felis eu sapien cursus vestibulum.,2,2019-12-02 17:38:27+00:00,2019-04-15 07:09:48+00:00
4,5,Tell me about the project you are most proud o...,Nam nulla.,3,2019-04-28 18:42:47+00:00,2019-05-02 23:23:41+00:00
...,...,...,...,...,...,...
85,86,tell me if this prompt is working for you guys,a test to see if we can all see the prompts/re...,127,2020-06-12 15:04:48.208000+00:00,2020-06-12 15:04:48.208000+00:00
86,87,"Looking back on the last 6 weeks, what advice ...",Just a quick piece of advice you may have.,134,2020-06-12 16:03:42.579000+00:00,2020-06-12 16:03:42.579000+00:00
87,88,"this is a prompt for chris, tell me if it's wo...",a test prompt,135,2020-06-12 19:31:03.466000+00:00,2020-06-12 19:31:03.466000+00:00
88,89,tell me your favorite color and what colors yo...,this is a test prompt for DS to get accurate v...,127,2020-06-12 22:37:42.831000+00:00,2020-06-12 22:37:42.831000+00:00


In [249]:
index +=1
table_name = db_table_names[index]

In [250]:
print(f"\nDATABASE TABLE: '{table_name.upper()}' \n")
get_db_table_as_dataframe(table_name)


DATABASE TABLE: 'TEAM_MEMBERS' 



,user_id,team_id,role_id
0,1,20,2
1,1,2,1
2,2,4,2
3,2,7,1
4,3,17,2
...,...,...,...
271,215,136,2
272,211,136,1
273,191,136,1
274,215,137,2


In [251]:
index +=1
table_name = db_table_names[index]

In [252]:
print(f"\nDATABASE TABLE: '{table_name.upper()}' \n")
get_db_table_as_dataframe(table_name)


DATABASE TABLE: 'TEAMS' 



,id,name,description,created_at,updated_at,avatar,organization_id,team_type
0,63,My personal team,My personal team,2020-04-28 20:42:47.312724+00:00,2020-04-28 20:42:47.312724+00:00,None,1,private
1,67,Superstars,Great team,2020-04-29 17:18:30.395687+00:00,2020-04-29 17:18:30.395687+00:00,None,3,private
2,70,General,This is a general team for all members,2020-04-29 17:57:02.223304+00:00,2020-04-29 17:57:02.223304+00:00,None,6,public
3,73,General,This is a general team for all members,2020-04-29 19:45:45.342592+00:00,2020-04-29 19:45:45.342592+00:00,None,8,public
4,74,Alpacavids,Alpacavids,2020-04-29 19:46:29.956266+00:00,2020-04-29 19:46:29.956266+00:00,None,7,private
...,...,...,...,...,...,...,...,...
130,124,My Team,Testing,2020-06-10 22:23:36.476054+00:00,2020-06-10 22:23:36.476054+00:00,None,38,private
131,128,last,test,2020-06-11 15:23:37.514021+00:00,2020-06-11 15:23:37.514021+00:00,None,39,private
132,131,test team,a test,2020-06-11 23:14:26.136109+00:00,2020-06-11 23:14:26.136109+00:00,None,36,private
133,134,TL Group,Hey!,2020-06-12 16:02:18.337136+00:00,2020-06-12 16:02:45.001000+00:00,None,43,private


In [253]:
index +=1
table_name = db_table_names[index]

In [254]:
print(f"\nDATABASE TABLE: '{table_name.upper()}' \n")
get_db_table_as_dataframe(table_name)


DATABASE TABLE: 'USERS' 



,id,email,username,password,first_name,last_name,avatar
0,2,oivashkin1@nsw.gov.au,tlefeaver1,$2b$08$IGM.eonWTK1wSIXyrBIo3eXaZTSXe3lu2haEHDO...,Leopold,Varndall,avatars/superhero-2.png
1,3,jmuldowney2@cargocollective.com,jdudney2,$2b$08$tXKxhKKZK5pkLyRQHY6EmOFlddr2GVWW/.anonP...,Brenna,Shwalbe,avatars/superhero-3.png
2,4,lgunner3@tamu.edu,tvolke3,$2b$08$T.xlpRKR11sqprkcyGo7xOcSmG///5Esy54aFDy...,Grace,Eyer,avatars/superhero-4.png
3,5,balderton4@printfriendly.com,cdeem4,$2b$08$fRxFtvmy3dzc1P5lLOkSH.H69QNmbjiGQDUXIsO...,Idaline,Gounet,avatars/superhero-5.png
4,6,aabazi5@nps.gov,nnosworthy5,$2b$08$26g1XtRPVEHjJlncwX3E6uuZvcaZG8s7D5l9IxS...,Sasha,Leys,avatars/superhero-6.png
...,...,...,...,...,...,...,...
199,225,teamreel@lambda.com,teamreel@lambda.com,$2b$08$7.O4yelSv9Tro85a0CqO5OcwRLHBOX/2E/ZiVKe...,Team,Reel,avatars/superhero-7.png
200,227,testaccount@test.com,testaccount@test.com,$2b$08$ay3fHHQa6XCuv84wUm3y.OJyPMRhfQnb6ueiwUY...,Test,Account,avatars/superhero-12.png
201,216,trollingyouhardly@gmail.com,trollingyouhardly@gmail.com,$2b$08$pYW5lsFhGnuoKzYJhVoHHu.9qsNCpiZQqF6XfQl...,jacob,clark,photos/ALPACAPIC-d1DYeJtBO.jpeg
202,226,naomipriest1604@gmail.com,naomipriest1604@gmail.com,$2b$08$kVeUSs4zZOT1pNYFsiEZWe9IyX6EwfeEwNnLQO8...,Naomi,Priest,photos/ALPACAPIC-FrimSe4hn.png


In [255]:
index +=1
table_name = db_table_names[index]

In [256]:
print(f"\nDATABASE TABLE: '{table_name.upper()}' \n")
get_db_table_as_dataframe(table_name)


DATABASE TABLE: 'VIDEOS' 



,id,owner_id,title,description,created_at,updated_at,video_url,prompt_id,thumbnail
0,156,227,tes,video,2020-06-11 16:23:43.657000+00:00,2020-06-11 16:23:43.657000+00:00,videos/ALPACAVID-Q8H6aTJWG.mp4,85,None
1,157,215,team reel test,a test video,2020-06-12 14:06:51.050000+00:00,2020-06-12 14:06:51.050000+00:00,videos/ALPACAVID-CLWz4H2Hk.mp4,85,None
2,158,216,heres the vid for chris,howdy,2020-06-12 14:19:24.658000+00:00,2020-06-12 14:19:24.658000+00:00,videos/ALPACAVID-LKxSoVSS1.mp4,85,None
3,159,226,working,it is,2020-06-12 15:06:04.780000+00:00,2020-06-12 15:06:04.780000+00:00,videos/ALPACAVID-5-F49OFyL.mp4,85,None
4,160,197,scotts response,a video,2020-06-12 16:36:36.912000+00:00,2020-06-12 16:36:36.912000+00:00,videos/ALPACAVID-EEEUEH_BW.mp4,85,None
5,167,198,Test,Just a short test video,2020-06-12 23:00:47.983000+00:00,2020-06-12 23:00:47.983000+00:00,videos/ALPACAVID-vKl_QByfJ.mp4,86,None
6,164,201,CAUSE BABY YOU'RE A FIIIIIIIREWORK,Come on let your colors burst. Make em go oh o...,2020-06-06 21:53:17.385000+00:00,2020-06-06 21:53:17.385000+00:00,videos/ALPACAVID-DLHzmth3D.mp4,85,None
7,165,201,Four score and seven years ago...,our fathers brought forth upon this continent ...,2020-06-11 22:00:31.719000+00:00,2020-06-11 22:00:31.719000+00:00,videos/ALPACAVID-BdHQPlY24.mp4,85,None
8,166,201,"If you like it, then you shoulda put a ring on...","Whoa oh oh, oh oh oh",2020-06-07 22:03:01.134000+00:00,2020-06-07 22:03:01.134000+00:00,videos/ALPACAVID-Q31886lax.mp4,85,None
9,161,201,DS test 002,DS test! 002,2020-06-08 21:46:35.033000+00:00,2020-06-08 21:46:35.033000+00:00,videos/ALPACAVID-uEBN2QTzz.mp4,85,None


In [257]:
index +=1
table_name = db_table_names[index]

In [258]:
print(f"\nDATABASE TABLE: '{table_name.upper()}' \n")
get_db_table_as_dataframe(table_name)


DATABASE TABLE: 'VIDEOS_FEEDBACK' 



,id,video_id,overall_performance,delivery_and_presentation,response_quality,audio_quality,visual_environment,attitude,sentiment_visual,sentiment_visual_details,...,background_visual_environment,background_noise,appearance_facial_centering,appearance_posture,appearance_gesticulation,human_overall_performance,human_delivery_and_presentation,human_response_quality,human_audio_quality,human_visual_environment
0,12,156,3.817,0.976,4.300,1.058,4.407,2.630,3.251,"{'emotions': {'sad': 0.4816375482887931, 'calm...",...,0.000,3.374,1.315,0.000,0.000,3.200,3.000,2.800,3.600,3.000
1,13,157,3.473,2.747,2.198,1.525,0.053,2.988,4.770,"{'emotions': {'sad': 0.9206712806497154, 'calm...",...,0.000,0.026,2.518,0.000,0.000,2.200,2.800,2.600,2.800,2.400
2,14,158,4.421,1.817,1.105,2.993,3.268,2.780,3.798,"{'emotions': {'sad': 0.02777736887348592, 'cal...",...,0.000,0.730,0.574,0.000,0.000,1.600,2.000,3.000,2.000,3.000
3,15,159,2.237,1.655,2.431,4.284,4.250,0.337,4.275,"{'emotions': {'sad': 0.1309930287853721, 'calm...",...,0.000,1.098,3.946,0.000,0.000,3.000,3.000,2.200,2.400,2.000
4,16,160,4.383,1.924,2.011,4.780,1.827,1.551,4.943,"{'emotions': {'sad': 0.9123695598577017, 'calm...",...,0.000,1.953,1.424,0.000,0.000,1.800,3.400,2.800,2.400,2.400
5,17,167,0.842,1.964,1.551,2.568,4.776,2.698,1.423,"{'emotions': {'sad': 0.810353040542465, 'calm'...",...,0.000,2.973,4.824,0.000,0.000,2.571,2.000,2.857,3.000,3.429
6,18,164,4.327,1.194,3.107,0.875,0.140,3.989,1.680,"{'emotions': {'sad': 0.21891780209641498, 'cal...",...,0.000,0.874,4.054,0.000,0.000,2.800,1.200,1.600,3.400,2.200
7,19,165,0.507,4.772,4.690,1.363,4.353,3.469,4.149,"{'emotions': {'sad': 0.385168748552734, 'calm'...",...,0.000,2.107,4.889,0.000,0.000,2.800,3.400,2.800,2.200,3.000
8,20,166,0.810,0.598,2.276,4.365,2.071,0.552,3.125,"{'emotions': {'sad': 0.40360854397311163, 'cal...",...,0.000,1.884,0.857,0.000,0.000,2.400,2.400,2.200,2.200,2.400
9,21,161,3.634,3.638,0.295,2.787,2.598,1.277,0.091,"{'emotions': {'sad': 0.7894593765567619, 'calm...",...,0.000,2.629,2.030,0.000,0.000,2.600,1.800,3.800,2.000,2.600


## **For Web: Queries for User-facing Charts:**

In [199]:
# Get a User's: 
# Overall Performance Score:

# # Query for Web/front-end:
# SELECT AVG(overall_performance)
# FROM (SELECT vf.overall_performance 
#       FROM videos_feedback as vf, videos as vds 
#       WHERE vf.video_id = vds.id 
#       AND vds.owner_id = {user_id}  
#       ORDER BY vds.created_at DESC 
#       LIMIT 3) 
#       AS recent_performance;

def get_overall_performance_score(user_id:int):
    query = f"SELECT AVG(overall_performance) FROM (SELECT vf.overall_performance FROM videos_feedback as vf, videos as vds WHERE vf.video_id = vds.id AND vds.owner_id = {user_id} ORDER BY vds.created_at DESC LIMIT 3) AS recent_performance;"
    pg_cursor.execute(query)
    results = pg_cursor.fetchall()[0][0]
    if results == None:
        return "No results: this user has not uploaded any videos."
    else:
        return float(results)

In [200]:
get_overall_performance_score(user_id=201)

2.0723333333333334

In [201]:
get_overall_performance_score(user_id=154)

'No results: this user has not uploaded any videos.'

In [202]:
# Get a User's: 
# Progress Over Time (Overall Performance Score Over Time):

# # Query for Web/front-end:
# SELECT * 
# FROM (SELECT vds.created_at, vf.overall_performance 
#       FROM videos_feedback as vf, videos as vds 
#       WHERE vf.video_id = vds.id 
#       AND vds.owner_id = 201  
#       ORDER BY vds.created_at DESC) 
#       AS performance_breakdown

def get_overall_performance_progress_time_series(user_id:int):
    query = f"SELECT * FROM (SELECT vds.created_at, vf.overall_performance FROM videos_feedback as vf, videos as vds WHERE vf.video_id = vds.id AND vds.owner_id = {user_id} ORDER BY vds.created_at DESC) AS performance_breakdown;"
    pg_cursor.execute(query)
    results = pg_cursor.fetchall()
    if results == []:
        return "No results: this user has not uploaded any videos."
    else:
        return results

In [203]:
get_overall_performance_progress_time_series(user_id=201)

[(datetime.datetime(2020, 6, 11, 22, 0, 31, 719000, tzinfo=psycopg2.tz.FixedOffsetTimezone(offset=0, name=None)),
  Decimal('0.507')),
 (datetime.datetime(2020, 6, 8, 21, 46, 43, 703000, tzinfo=psycopg2.tz.FixedOffsetTimezone(offset=0, name=None)),
  Decimal('2.076')),
 (datetime.datetime(2020, 6, 8, 21, 46, 35, 33000, tzinfo=psycopg2.tz.FixedOffsetTimezone(offset=0, name=None)),
  Decimal('3.634')),
 (datetime.datetime(2020, 6, 7, 22, 3, 1, 134000, tzinfo=psycopg2.tz.FixedOffsetTimezone(offset=0, name=None)),
  Decimal('0.810')),
 (datetime.datetime(2020, 6, 6, 21, 53, 17, 385000, tzinfo=psycopg2.tz.FixedOffsetTimezone(offset=0, name=None)),
  Decimal('4.327')),
 (datetime.datetime(2020, 6, 5, 21, 47, 50, 147000, tzinfo=psycopg2.tz.FixedOffsetTimezone(offset=0, name=None)),
  Decimal('2.656'))]

In [204]:
get_overall_performance_progress_time_series(user_id=154)

'No results: this user has not uploaded any videos.'

In [205]:
# Get a User's: 
# Performance Breakdown (Scores in Each Sub-area for Front-end Charts):

# # Query for Web/front-end:
# SELECT AVG(human_delivery_and_presentation) AS human_delivery_and_presentation, 
# AVG(human_response_quality) AS human_response_quality, 
# AVG(human_audio_quality) AS human_audio_quality, 
# AVG(human_visual_environment) AS human_visual_environment, 
# AVG(attitude) AS attitude, 
# AVG(speaking_speed) AS speaking_speed, 
# AVG(background_noise) AS background_noise, 
# AVG(appearance_facial_centering) AS appearance_facial_centering
# FROM (SELECT vf.human_delivery_and_presentation, vf.human_response_quality, 
#       vf.human_audio_quality, vf.human_visual_environment, vf.attitude, 
#       vf.speaking_speed, vf.background_noise, vf.appearance_facial_centering, 
#       vf.appearance_posture 
#       FROM videos_feedback as vf, videos as vds 
#       WHERE vf.video_id = vds.id 
#       AND vds.owner_id = {user_id} 
#       ORDER BY vds.created_at DESC 
#       LIMIT 3) 
#       AS performance_breakdown;

def get_breakdown_scores(user_id:int):
    query = f"SELECT AVG(human_delivery_and_presentation) AS human_delivery_and_presentation, AVG(human_response_quality) AS human_response_quality, AVG(human_audio_quality) AS human_audio_quality, AVG(human_visual_environment) AS human_visual_environment, AVG(attitude) AS attitude, AVG(speaking_speed) AS speaking_speed, AVG(background_noise) AS background_noise, AVG(appearance_facial_centering) AS appearance_facial_centering FROM (SELECT vf.human_delivery_and_presentation, vf.human_response_quality, vf.human_audio_quality, vf.human_visual_environment, vf.attitude, vf.speaking_speed, vf.background_noise, vf.appearance_facial_centering, vf.appearance_posture FROM videos_feedback as vf, videos as vds WHERE vf.video_id = vds.id AND vds.owner_id = {user_id}  ORDER BY vds.created_at DESC LIMIT 3) AS performance_breakdown;"
    pg_cursor.execute(query)
    results = pg_cursor.fetchall()[0]
    if None in results:
        return [item for item in results]
    else:
        return [float(item) for item in results]

In [206]:
get_breakdown_scores(user_id=201)

[2.7333333333333334,
 3.2666666666666666,
 2.3333333333333335,
 2.6,
 2.0456666666666665,
 2.737,
 2.029,
 3.451]

In [207]:
get_breakdown_scores(user_id=154)

[None, None, None, None, None, None, None, None]

## **Functions to Add Fake Data to the DB for Testing:**

In [49]:
# import datetime
# import numpy as np
# import copy
# import json

In [50]:
# FAKE_INPUT_USER_PERFORMANCE = {"user_id": 227}

# FAKE_INPUT_PROMPT_TOP_RESPONSES = {"prompt_id": 85}

# FAKE_INPUT_VIDEO_ANALYSIS = {"video_id": 156}

In [51]:
# # 'videos' TABLE: CHANGE DATED CREATED SO THEY'RE SPREAD OUT ON CHARTS (NOT ALL THE SAME DAY):

# # Show all tables in our Postgres DB:
# pg_cursor.execute("SELECT title FROM videos WHERE owner_id = 201")

# titles = pg_cursor.fetchall()

# # -------------------------------------------------------------------

# for item in titles:
#     title = item[0]
    
#     # Show all tables in our Postgres DB:
#     pg_cursor.execute("SELECT * FROM videos WHERE title = (%s)", (title,))

#     test21 = pg_cursor.fetchall()
    
#     created = test21[0][4]
#     updated = test21[0][5]
    
#     new_day = np.random.randint(4, 12)
#     created = created.replace(day=new_day)
#     updated = updated.replace(day=new_day)
    
#     query_change = "UPDATE videos SET created_at = %(created)s, updated_at = %(updated)s WHERE title = %(title)s"
    
#     # Show all tables in our Postgres DB:
#     pg_cursor.execute(query_change, 
#                       {'created': created, 'updated': updated, 'title': title}
#                      )

#     pg_conn.commit()

In [52]:
# # 'feedback' TABLE: POPULATE WITH FAKE FEEDBACK DATA:

# # Get all video_ids and their owner_ids from 'videos' table:
# pg_cursor.execute("SELECT id FROM videos")
# results = pg_cursor.fetchall()

# # For each video in our DB (in the 'videos' table):
# for tuple in results:
#     video_id = tuple[0]
    
#     # Users on Team 127 (we only want feedback from them right now for testing):
#     pg_cursor.execute("SELECT user_id FROM team_members WHERE team_id = 127;")
#     results = pg_cursor.fetchall()
#     users_on_team_127 = [row[0] for row in results]
    
#     # Add 5 reviews ("feedbacks") in the 'feedback' table:
#     for int in range(5):    
#         random_day = np.random.randint(2, 12)
#         created_at = date_sample.replace(day=random_day)
#         updated_at = date_sample.replace(day=random_day)
        
#         random_user_on_team_127 = users_on_team_127[np.random.randint(0, 10)]

#         pg_cursor.execute("""INSERT INTO feedback (post, video_id, owner_id, created_at, updated_at, viewed, overall_performance, delivery_and_presentation, response_quality, audio_quality, visual_environment) 
#                           VALUES (%(post)s, %(video_id)s, %(owner_id)s, %(created_at)s, %(updated_at)s, %(viewed)s, %(overall_performance)s, %(delivery_and_presentation)s, %(response_quality)s, %(audio_quality)s, %(visual_environment)s)""", 
#                          {
#                              "post": "Great -- no comments!", 
#                              "video_id": video_id, 
#                              "owner_id": random_user_on_team_127, 
#                              "created_at": created_at, "updated_at": updated_at, 
#                              "viewed": False, 
#                              "overall_performance": np.random.randint(1, 5), 
#                              "delivery_and_presentation": np.random.randint(1, 5), 
#                              "response_quality": np.random.randint(1, 5), 
#                              "audio_quality": np.random.randint(1, 5), 
#                              "visual_environment": np.random.randint(1, 5)
#                          }
#                          )

# pg_conn.commit()

In [53]:
# # 'videos_feedback' TABLE: POPULATE WITH FAKE DATA:

# # Get all video_ids and their owner_ids from 'videos' table:
# pg_cursor.execute("SELECT id, owner_id FROM videos")
# results = pg_cursor.fetchall()

# # For each video in our DB (in the 'videos' table):
# for tuple in results:
#     video_id = tuple[0]
#     owner_id = tuple[1]
    
#     sentiment_visual_details = {
#         'emotions': {
#             'calm': np.random.uniform(0, 1), 
#             'surprised': np.random.uniform(0, 1), 
#             'disgusted': np.random.uniform(0, 1), 
#             'angry': np.random.uniform(0, 1), 
#             'sad': np.random.uniform(0, 1), 
#             'confused': np.random.uniform(0, 1), 
#             'happy': np.random.uniform(0, 1), 
#             'fear': np.random.uniform(0, 1)
#         }
#     }
    
#     sentiment_audio_details = {
#         'positive': np.random.uniform(0, 1), 
#         'neutral': np.random.uniform(0, 1), 
#         'negative': np.random.uniform(0, 1)
#     }
    
#     # Get all human feedback on that video from the 'feedback' table:
#     feedback_on_video = get_feedback_for_video(video_id=video_id)
    
#     # Add fake feedback for that video into the 'video_feedback' table in our DB:
#     pg_cursor.execute("""INSERT INTO videos_feedback (video_id, overall_performance, delivery_and_presentation, response_quality, audio_quality, visual_environment, attitude, sentiment_visual, sentiment_visual_details, sentiment_audio, sentiment_audio_details, speaking_confidence, speaking_volume, speaking_vocabulary, speaking_speed, speaking_filler_words, background_visual_environment, background_noise, appearance_facial_centering, appearance_posture, appearance_gesticulation, human_overall_performance, human_delivery_and_presentation, human_response_quality, human_audio_quality, human_visual_environment) 
#                       VALUES (%(video_id)s, %(overall_performance)s, %(delivery_and_presentation)s, %(response_quality)s, %(audio_quality)s, %(visual_environment)s, %(attitude)s, %(sentiment_visual)s, %(sentiment_visual_details)s, %(sentiment_audio)s, %(sentiment_audio_details)s, %(speaking_confidence)s, %(speaking_volume)s, %(speaking_vocabulary)s, %(speaking_speed)s, %(speaking_filler_words)s, %(background_visual_environment)s, %(background_noise)s, %(appearance_facial_centering)s, %(appearance_posture)s, %(appearance_gesticulation)s, %(human_overall_performance)s, %(human_delivery_and_presentation)s, %(human_response_quality)s, %(human_audio_quality)s, %(human_visual_environment)s)""", 
#                      {
#                          'video_id': video_id, 
#                          'overall_performance': np.random.uniform(0, 5), 
#                          'delivery_and_presentation': np.random.uniform(0, 5), 
#                          'response_quality': np.random.uniform(0, 5), 
#                          'audio_quality': np.random.uniform(0, 5), 
#                          'visual_environment': np.random.uniform(0, 5), 
#                          'attitude': np.random.uniform(0, 5), 
#                          'sentiment_visual': np.random.uniform(0, 5), 
#                          'sentiment_visual_details': json.dumps(sentiment_visual_details), 
#                          'sentiment_audio': np.random.uniform(0, 5), 
#                          'sentiment_audio_details': json.dumps(sentiment_audio_details), 
#                          'speaking_confidence': 0, 
#                          'speaking_volume': 0, 
#                          'speaking_vocabulary': np.random.uniform(0, 5), 
#                          'speaking_speed': np.random.uniform(0, 5), 
#                          'speaking_filler_words': 0, 
#                          'background_visual_environment': 0, 
#                          'background_noise': np.random.uniform(0, 5), 
#                          'appearance_facial_centering': np.random.uniform(0, 5), 
#                          'appearance_posture': 0, 
#                          'appearance_gesticulation': 0, 
#                          'human_overall_performance': feedback_on_video['overall_performance'].mean(), 
#                          'human_delivery_and_presentation': feedback_on_video['delivery_and_presentation'].mean(), 
#                          'human_response_quality': feedback_on_video['response_quality'].mean(), 
#                          'human_audio_quality': feedback_on_video['audio_quality'].mean(), 
#                          'human_visual_environment': feedback_on_video['visual_environment'].mean()
#                      }
#                      )

# pg_conn.commit()

In [54]:
# # Replace feedback-givers (owner_id in the 'feedback' table) with only people from our testing team, team_id #127:

# # Users on Team 127 (we only want feedback from them right now for testing):
# pg_cursor.execute("SELECT user_id FROM team_members WHERE team_id = 127;")
# results = pg_cursor.fetchall()
# users_on_team_127 = [row[0] for row in results]

# pg_cursor.execute("SELECT id FROM feedback")
# results = pg_cursor.fetchall()
# feedback_ids = [tuple[0] for tuple in results]

# for id in feedback_ids:
#     random_user_on_team_127 = users_on_team_127[np.random.randint(0, 10)]
#     print(f"id: {id}, reviewer: {random_user_on_team_127}")
    
#     pg_cursor.execute("UPDATE feedback SET owner_id = %(user)s WHERE id = %(id)s", 
#                       {'user': random_user_on_team_127, 
#                        'id': id})

# pg_conn.commit()

## **Close the Cursor and Connection:**

In [105]:
# Close the cursor:
pg_cursor.close()

In [106]:
# Close the connection:
pg_conn.close()